# Ramsey King
# DSC 540 - Data Preparation
# Weeks 5 & 6
# July 16, 2021
## For this assignment you need to complete 8 of the following exercises against this data.  You must select at least 2 methods from Chapters 7, 8, 10 & 11.

In [108]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

candy_2016_df = pd.read_excel('BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx')
candy_2016_df.shape

(1259, 123)

### The 2 method from Chapter 7 that I will chose are filtering out missing data and binning. This will be done by filtering out the rows where the column 'How old are you?' is non-numeric.  From there, the ages will be binned.

In [86]:
# rename the 'How old are you?' column to 'age'
candy_2016_df = candy_2016_df.rename(columns={candy_2016_df.columns[3]:'age'})

# remove rows where 'age' column does not contain a number
candy_2016_df = candy_2016_df[pd.to_numeric(candy_2016_df['age'], errors='coerce').notnull()]
candy_2016_df.shape

(1191, 123)

In [87]:
# bin 'age' column based on age ranges (<17, 18-25, 26-34, 35-44, 45-54, 55-64, >65)
bins = [0,17,24,34,44,54,64,140]
cats = pd.cut(candy_2016_df['age'], bins)
pd.value_counts(cats)
# candy_2016_df.plot.bar(x=cats,y=pd.value_counts(cats))

(34, 44]     382
(24, 34]     317
(44, 54]     292
(54, 64]     121
(17, 24]      37
(0, 17]       21
(64, 140]     19
Name: age, dtype: int64

### The 2 methods that I will chose from Chapter 8 are combining/merging datasets and pivoting data.  This will be acheived as follows:
1. Create a subset of the 2016 and 2017 candy datasets based on all participants who gave a response in the '[100 Grand Bar]' column
2. Merge these subset datasets together (adding the rows of the 2016 dataset to the 2017 dataset).
3. Creating a pivot table based on the 100 Grand Bar response type by gender

In [88]:
# Load the 2016 and 2017 datasets (since I've re-written over the 2016 dataset above)

candy_2016_df = pd.read_excel('BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx')
candy_2017_df = pd.read_excel('candyhierarchy2017.xlsx')

print("2016 dimensions:", candy_2016_df.shape, "\n2017 dimensions:", candy_2017_df.shape)

2016 dimensions: (1259, 123) 
2017 dimensions: (2460, 120)


In [89]:
# rename the '100 Grand Bar' column '100_grand_bar', 'Your gender:' column 'gender', and 'How old are you?' 'age'
candy_2016_df = candy_2016_df.rename(columns={candy_2016_df.columns[2]:'gender',
                                              candy_2016_df.columns[3]:'age',
                                              candy_2016_df.columns[6]:'hundred_grand_bar'})
#candy_2016_df.columns


# check for empty values in the hundred_grand_bar column in the 2016 dataset.
miss = candy_2016_df['hundred_grand_bar'].isnull().sum()
if miss>0:
    print("hundred_grand_bar has {} missing value(s)".format(miss))
else:
    print("hundred_grand_bar has no missing values.")


hundred_grand_bar has 78 missing value(s)


In [90]:
# remove rows where the 100 Grand Bar value is null
candy_2016_df = candy_2016_df[candy_2016_df['hundred_grand_bar'].isnull() == False]
candy_2016_df.shape

(1181, 123)

In [91]:
# rename the '100 Grand Bar' column '100_grand_bar', 'Your gender:' column 'gender', and 'How old are you?' 'age'
candy_2017_df = candy_2017_df.rename(columns={candy_2017_df.columns[2]:'gender',
                                              candy_2017_df.columns[3]:'age',
                                              candy_2017_df.columns[6]:'hundred_grand_bar'})
#candy_2017_df.columns


# check for empty values in the hundred_grand_bar column in the 2016 dataset.
miss = candy_2017_df['hundred_grand_bar'].isnull().sum()
if miss>0:
    print("hundred_grand_bar has {} missing value(s)".format(miss))
else:
    print("hundred_grand_bar has no missing values.")

hundred_grand_bar has 747 missing value(s)


In [92]:
# remove rows where the 100 Grand Bar value is null
candy_2017_df = candy_2017_df[candy_2017_df['hundred_grand_bar'].isnull() == False]
candy_2017_df.shape

(1713, 120)

In [106]:
'''
merge the 2016 and 2017 candy datasets together by adding rows.  Before we do this, we will grab only the gender, age,
and hundred_grand_bar columns and put them into a new dataframe
'''
candy_2016_subset = candy_2016_df[['gender', 'age', 'hundred_grand_bar']]
candy_2017_subset = candy_2017_df[['gender', 'age', 'hundred_grand_bar']]

combined_candy_subset = candy_2016_subset.append(candy_2017_subset)
combined_candy_subset

,gender,age,hundred_grand_bar
0,Male,22,JOY
1,Male,45,MEH
2,Female,48,JOY
3,Male,57,JOY
4,Male,42,MEH


In [117]:
'''
Now we are ready to pivot the data based on gender.
'''
combined_candy_subset.dtypes
# drop the rows with missing values for age
combined_candy_subset = combined_candy_subset[combined_candy_subset['age'].isnull() == False]
print(combined_candy_subset)
# combined_candy_subset = combined_candy_subset[combined_candy_subset['age'].str.isnumeric() == True]
#
# # convert age column to numeric
# pd.to_numeric(combined_candy_subset['age'])
# combined_candy_subset.pivot_table(values=['age'], index=['hundred_grand_bar'],aggfunc=np.mean)



Empty DataFrame
Columns: [gender, age, hundred_grand_bar]
Index: []
